# Install requirements

In [ ]:
!pip install Datasets
!pip install transformers
!pip install scikit-learn

In [ ]:
import pandas as pd
import numpy as np
from datasets import Dataset
from transformers import AutoTokenizer
import tensorflow as tf
from sklearn.model_selection import train_test_split
from transformers import TFAutoModelForSequenceClassification
from tensorflow.keras.optimizers import Adam
import datasets

### Use for debug only

In [ ]:
tf.config.run_functions_eagerly(True)
tf.data.experimental.enable_debug_mode()

In [ ]:
def preprocess_function(examples, tokenizer, max_length=128):
    return tokenizer(examples["text"],
                     truncation=True,
                     padding='max_length',
                     max_length=max_length,
                     return_tensors="np")

In [ ]:
def get_data(train_path):
   train_df = pd.read_json(train_path, lines=True)
   train_df = train_df[['text', 'label']]
   train_df, test_df = train_test_split(train_df, test_size=0.2, stratify=train_df['label'], random_state=42)
   return train_df, test_df


In [ ]:
file_path = 'data/subtaskA_train_monolingual.jsonl'

In [ ]:
train_df, test_df = get_data(file_path)

In [ ]:
print(f"Train shape: {train_df.shape}")
print(f"Test shape: {test_df.shape}")

In [ ]:
# take only 100 sample from training data (test purposes)
train_df = train_df[:100]
test_df = test_df[:100]

In [ ]:
labels = np.asarray(train_df['label'])

In [ ]:
id2label = {0: "human", 1: "machine"}
label2id = {"human": 0, "machine": 1 }

In [ ]:
# pandas dataframe to huggingface Dataset
train_dataset= Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

In [ ]:
print(train_dataset)

In [ ]:
dataset = datasets.DatasetDict({'train': train_dataset, 'test': test_dataset})

In [ ]:
print(dataset)

In [ ]:
dataset["train"][0]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

In [ ]:
tokenizer("Hello, this is a sentence!", "And this sentence goes with it.")

In [ ]:
# pre_tokenizer_columns = set(dataset["train"].features)
encoded_dataset = dataset.map(preprocess_function, batched=True, fn_kwargs={'tokenizer': tokenizer})
# tokenizer_columns = list(set(encoded_dataset["train"].features) - pre_tokenizer_columns)
# print("Columns added by tokenizer:", tokenizer_columns)

In [ ]:
encoded_dataset["train"].features["label"]

In [ ]:
model = TFAutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=len(label2id), id2label=id2label, label2id=label2id)

In [ ]:
tf_train_dataset = model.prepare_tf_dataset(encoded_dataset["train"],
                                      batch_size=8,
                                      shuffle=True,
                                      tokenizer=tokenizer)

In [ ]:
tf_test_dataset = model.prepare_tf_dataset(encoded_dataset["train"],
                                          batch_size=8,
                                          shuffle=False,
                                          tokenizer=tokenizer)

In [ ]:
model.compile(optimizer=Adam(3e-5), metrics=['accuracy'])  # No loss argument!

In [ ]:
model.fit(tf_train_dataset)